In [17]:
from ultralytics import YOLO
import cv2
import numpy as np


front_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg'
back_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg'
model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
model = YOLO(model_path)

def predict(image_path):
    results = model.predict(image_path, save=True)
    fruits = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        class_name = model.names[int(box.cls)]        
        # Calculate center point
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        
        fruits.append({
            "bbox": (x1, y1, x2, y2),
            "center": (center_x, center_y),
            "class": class_name
        })
    
    return fruits

# Step 3: Simple color detection (RGB-based)
def get_color(roi):
    avg_color = np.mean(roi, axis=(0, 1))  # Get average RGB values
    r, g, b = avg_color
    
    # Simple color thresholds (adjust these as needed)
    if r > g and r > b: return "red"
    elif g > r and g > b: return "green"
    elif b > r and b > g: return "blue"
    else: return "unknown"

# Step 4: Process images
front_img = cv2.imread(front_image)
back_img = cv2.imread(back_image)

front_fruits = predict(front_image)
back_fruits = predict(back_image)

# Step 5: Match fruits between front and back
counts = {"red": 0, "green": 0, "blue": 0, "unknown": 0}

# Check front fruits
for front in front_fruits:
    x1, y1, x2, y2 = front["bbox"]
    roi = front_img[y1:y2, x1:x2]
    color = get_color(roi)
    
    # Check if same fruit exists in back image
    is_duplicate = False
    for back in back_fruits:
        # Simple distance check (adjust 50 as needed)
        distance = np.sqrt((front["center"][0]-back["center"][0])**2 + 
                          (front["center"][1]-back["center"][1])**2)
        
        if distance < 50:  # If centers are close
            is_duplicate = True
            break
    
    if not is_duplicate:
        counts[color] += 1

# Check remaining back fruits
for back in back_fruits:
    x1, y1, x2, y2 = back["bbox"]
    roi = back_img[y1:y2, x1:x2]
    color = get_color(roi)
    counts[color] += 1

# Step 6: Show results
print(front_fruits)
print("Final counts:")
for color, count in counts.items():
    print(f"{color}: {count}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg: 480x640 1 2, 1 3, 575.2ms
Speed: 8.0ms preprocess, 575.2ms inference, 7.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict16

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg: 480x640 1 3, 561.0ms
Speed: 5.5ms preprocess, 561.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict16
[{'bbox': (360, 202, 462, 291), 'center': (411, 246), 'class': '3'}, {'bbox': (404, 240, 421, 271), 'center': (412, 255), 'class': '2'}]
Final counts:
red: 1
green: 2
blue: 0
unknown: 0


In [27]:
import shutil

# Zip the entire runs directory
shutil.make_archive('runs', 'zip', 'runs')
print("Runs directory zipped as runs.zip")
from IPython.display import FileLink

# Create a download link for the zipped file
FileLink('runs.zip')


Runs directory zipped as runs.zip


C:\Users\saart\runs.zip

In [15]:
from ultralytics import YOLO
import cv2
import numpy as np

# Paths configuration
front_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg'
back_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg'
model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'

# Initialize YOLO model
model = YOLO(model_path)

def predict(image_path):
    """Detect fruits using YOLO and return their information"""
    results = model.predict(image_path, conf=0.5)  # Increased confidence threshold
    fruits = []
    
    for box in results[0].boxes:
        if box.conf < 0.5:  # Skip low-confidence detections
            continue
            
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        
        fruits.append({
            "bbox": (x1, y1, x2, y2),
            "center": (center_x, center_y)
        })
    
    return fruits

def get_color(roi):
    """Detect color using HSV color space"""
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    avg_hue = np.mean(hsv[:, :, 0])
    
    # Define HSV color ranges (Hue, Saturation, Value)
    if 0 <= avg_hue < 15 or 160 <= avg_hue <= 180:
        return "red"
    elif 35 <= avg_hue < 85:
        return "green"
    elif 85 <= avg_hue < 100:
        return "blue"
    elif 100 <= avg_hue < 160:
        return "purple"
    return "unknown"

def align_images(front_img, back_img):
    """Align images using feature matching and return homography matrix"""
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

# Load images
front_img = cv2.imread(front_image)
back_img = cv2.imread(back_image)

# Get fruit detections
front_fruits = predict(front_image)
back_fruits = predict(back_image)

# Align images using homography
H = align_images(front_img, back_img)

# Initialize counters
color_counts = {"red": 0, "green": 0, "blue": 0, "purple": 0, "unknown": 0}
matched_pairs = set()

# Process front image fruits
for idx, front in enumerate(front_fruits):
    x1, y1, x2, y2 = front["bbox"]
    roi = front_img[y1:y2, x1:x2]
    color = get_color(roi)
    
    # Check for matches in back image
    for back_idx, back in enumerate(back_fruits):
        # Transform back fruit center to front image coordinates
        back_center = np.array([[back["center"]]], dtype=np.float32)
        transformed_center = cv2.perspectiveTransform(back_center, H)[0][0]
        
        # Calculate distance in aligned coordinates
        distance = np.linalg.norm(np.array(front["center"]) - transformed_center)
        
        if distance < 30:  # Match threshold in pixels
            matched_pairs.add(back_idx)
            # Take color from both views for verification
            back_roi = back_img[back["bbox"][1]:back["bbox"][3], back["bbox"][0]:back["bbox"][2]]
            back_color = get_color(back_roi)
            
            if color == back_color:
                color_counts[color] += 1
            break
    else:
        color_counts[color] += 1

# Process unmatched back fruits
for idx, back in enumerate(back_fruits):
    if idx not in matched_pairs:
        roi = back_img[back["bbox"][1]:back["bbox"][3], back["bbox"][0]:back["bbox"][2]]
        color = get_color(roi)
        color_counts[color] += 1

# Display results
print("Final Fruit Counts:")
for color, count in color_counts.items():
    print(f"{color.capitalize()}: {count}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg: 480x640 1 2, 1 3, 675.1ms
Speed: 10.6ms preprocess, 675.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg: 480x640 1 3, 632.3ms
Speed: 7.7ms preprocess, 632.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Final Fruit Counts:
Red: 0
Green: 2
Blue: 0
Purple: 0
Unknown: 0


In [13]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(image_path, model):
    """Detect fruits and return bounding boxes, centers, and confidence scores"""
    results = model.predict(image_path, conf=0.5)[0]
    detections = []
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = (int((x1 + x2)/2), int((y1 + y2)/2))
        detections.append({
            'bbox': (x1, y1, x2, y2),
            'center': center,
            'conf': float(box.conf)
        })
    
    return detections

def get_color(img, bbox):
    """Get dominant color of fruit using HSV color space"""
    x1, y1, x2, y2 = bbox
    roi = img[y1:y2, x1:x2]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Get average HSV values
    avg_hsv = np.mean(hsv, axis=(0, 1))
    
    # More robust color ranges
    if (avg_hsv[0] < 15 or avg_hsv[0] > 160) and avg_hsv[1] > 50:
        return "red"
    elif 35 <= avg_hsv[0] < 85 and avg_hsv[1] > 30:
        return "green"
    elif 85 <= avg_hsv[0] < 125 and avg_hsv[1] > 30:
        return "blue"
    elif 125 <= avg_hsv[0] < 160 and avg_hsv[1] > 30:
        return "purple"
    return "unknown"

def compute_homography(front_img, back_img):
    """Compute homography matrix between two images"""
    # Use ORB feature detector (faster than SIFT)
    orb = cv2.ORB_create(nfeatures=2000)
    
    # Find keypoints and descriptors
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    if des1 is None or des2 is None:
        return None
        
    # Match features
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    
    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)
    
    # Use best matches for homography
    good_matches = matches[:min(50, len(matches))]
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    # Compute homography
    H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    
    return H

def count_fruits(front_path, back_path, model_path):
    """Count fruits while avoiding double counting between views"""
    # Initialize
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    color_counts = {"red": 0, "green": 0, "blue": 0, "purple": 0, "unknown": 0}
    
    # Get detections
    front_detections = detect_fruits(front_path, model)
    back_detections = detect_fruits(back_path, model)
    
    # Compute homography
    H = compute_homography(front_img, back_img)
    
    if H is None:
        print("Warning: Could not compute homography. Using single view counting.")
        # Fall back to counting only front view
        for det in front_detections:
            color = get_color(front_img, det['bbox'])
            color_counts[color] += 1
        return color_counts
    
    # Transform back centers to front image space
    back_centers = np.float32([d['center'] for d in back_detections]).reshape(-1, 1, 2)
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2)
    
    # Track matched fruits
    matched_back_indices = set()
    
    # Process front detections first
    for front_det in front_detections:
        front_color = get_color(front_img, front_det['bbox'])
        front_center = np.array(front_det['center'])
        
        # Look for matches in back view
        best_match = None
        min_dist = 50  # Pixel threshold for matching
        
        for i, back_center in enumerate(transformed_centers):
            if i in matched_back_indices:
                continue
                
            dist = np.linalg.norm(front_center - back_center)
            if dist < min_dist:
                back_color = get_color(back_img, back_detections[i]['bbox'])
                # Verify match with color and confidence
                if back_color == front_color:
                    min_dist = dist
                    best_match = i
        
        if best_match is not None:
            # Found a match - count only once
            matched_back_indices.add(best_match)
            color_counts[front_color] += 1
        else:
            # No match found - count as new fruit
            color_counts[front_color] += 1
    
    # Count unmatched fruits from back view
    for i, back_det in enumerate(back_detections):
        if i not in matched_back_indices:
            color = get_color(back_img, back_det['bbox'])
            color_counts[color] += 1
    
    return color_counts

# Example usage
if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    counts = count_fruits(front_path, back_path, model_path)
    print("\nFinal Fruit Counts:")
    for color, count in counts.items():
        print(f"{color.capitalize()}: {count}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg: 480x640 1 2, 1 3, 524.1ms
Speed: 4.7ms preprocess, 524.1ms inference, 7.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg: 480x640 1 3, 427.5ms
Speed: 0.0ms preprocess, 427.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

Final Fruit Counts:
Red: 0
Green: 2
Blue: 0
Purple: 0
Unknown: 0


In [23]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(image_path, model):
    """Detect fruits and return their locations"""
    results = model.predict(image_path)[0]
    detections = []
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = (int((x1 + x2)/2), int((y1 + y2)/2))
        detections.append({
            'bbox': (x1, y1, x2, y2),
            'center': center
        })
    
    return detections

def get_color(img, bbox):
    """Get fruit color from bbox region"""
    x1, y1, x2, y2 = bbox
    roi = img[y1:y2, x1:x2]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    avg_hsv = np.mean(hsv, axis=(0, 1))
    
    if avg_hsv[0] < 15 or avg_hsv[0] > 160: return "red"
    if 35 <= avg_hsv[0] < 85: return "green"
    if 85 <= avg_hsv[0] < 125: return "blue"
    if 125 <= avg_hsv[0] < 160: return "purple"
    return "unknown"

def match_and_count_fruits(front_path, back_path, model_path):
    """Count fruits while avoiding double counting"""
    # Initialize
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    color_counts = {"red": 0, "green": 0, "blue": 0, "purple": 0, "unknown": 0}
    
    # Get detections
    front_dets = detect_fruits(front_path, model)
    back_dets = detect_fruits(back_path, model)
    
    # Get colors for all detections
    front_colors = [get_color(front_img, d['bbox']) for d in front_dets]
    back_colors = [get_color(back_img, d['bbox']) for d in back_dets]
    
    # Match fruits between views using color and position
    matched_back = set()
    
    for i, (front_det, front_color) in enumerate(zip(front_dets, front_colors)):
        front_center = np.array(front_det['center'])
        matched = False
        
        # Try to find a matching fruit in back view
        for j, (back_det, back_color) in enumerate(zip(back_dets, back_colors)):
            if j in matched_back:
                continue
                
            if front_color == back_color:
                back_center = np.array(back_det['center'])
                # Use simple distance threshold for matching
                if np.linalg.norm(front_center - back_center) < 100:  # Adjust threshold if needed
                    matched_back.add(j)
                    matched = True
                    break
        
        # Count the fruit
        if matched:
            color_counts[front_color] += 1
        else:
            color_counts[front_color] += 1
    
    # Count remaining unmatched fruits from back view
    for j, back_color in enumerate(back_colors):
        if j not in matched_back:
            color_counts[back_color] += 1
    
    return color_counts

def main():
    # Replace with your paths
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    counts = match_and_count_fruits(front_path, back_path, model_path)
    
    print("\nFruit Counts:")
    for color, count in counts.items():
        if count > 0:  # Only show colors that were found
            print(f"{color.capitalize()}: {count}")

if __name__ == "__main__":
    main()


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg: 480x640 1 2, 1 3, 564.9ms
Speed: 4.0ms preprocess, 564.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg: 480x640 1 3, 504.8ms
Speed: 9.8ms preprocess, 504.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

Fruit Counts:
Green: 2


In [29]:
from ultralytics import YOLO
import cv2
import numpy as np

def predict(image_path, model):
    """Detect fruits using YOLO and return their information"""
    results = model.predict(image_path)[0]
    fruits = []
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        
        fruits.append({
            "bbox": (x1, y1, x2, y2),
            "center": (center_x, center_y)
        })
    
    return fruits

def get_color(img, bbox):
    """Detect color using HSV color space"""
    x1, y1, x2, y2 = bbox
    roi = img[y1:y2, x1:x2]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    avg_hue = np.mean(hsv[:, :, 0])
    
    if 0 <= avg_hue < 15 or 160 <= avg_hue <= 180:
        return "red"
    elif 35 <= avg_hue < 85:
        return "green"
    elif 85 <= avg_hue < 100:
        return "blue"
    elif 100 <= avg_hue < 160:
        return "purple"
    return "unknown"

def align_images(front_img, back_img):
    """Align images using feature matching and return homography matrix"""
    orb = cv2.ORB_create(nfeatures=3000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    
    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)
    
    # Use best matches
    good_matches = matches[:min(len(matches), 50)]
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_fruits(front_path, back_path, model_path):
    """Main function to count fruits with homography-based matching"""
    # Initialize
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    # Get detections
    front_fruits = predict(front_path, model)
    back_fruits = predict(back_path, model)
    
    # Compute homography matrix
    H = align_images(front_img, back_img)
    
    # Initialize counters
    color_counts = {"red": 0, "green": 0, "blue": 0, "purple": 0, "unknown": 0}
    matched_back_indices = set()
    
    # Process each back fruit
    for back_idx, back_fruit in enumerate(back_fruits):
        back_x, back_y = back_fruit["center"]
        # Transform back coordinates to front view
        transformed_point = cv2.perspectiveTransform(
            np.array([[[back_x, back_y]]], dtype=np.float32), H
        )
        tx, ty = transformed_point[0][0]
        
        # Get color of back fruit
        back_color = get_color(back_img, back_fruit["bbox"])
        matched = False
        
        # Compare with front fruits
        for front_idx, front_fruit in enumerate(front_fruits):
            front_x, front_y = front_fruit["center"]
            distance = np.sqrt((front_x - tx)**2 + (front_y - ty)**2)
            
            if distance < 30:  # Threshold in aligned coordinates
                front_color = get_color(front_img, front_fruit["bbox"])
                if front_color == back_color:
                    matched = True
                    matched_back_indices.add(back_idx)
                    color_counts[front_color] += 1
                    break
        
        if not matched:
            color_counts[back_color] += 1
    
    # Count unmatched front fruits
    for front_fruit in front_fruits:
        front_color = get_color(front_img, front_fruit["bbox"])
        front_x, front_y = front_fruit["center"]
        matched = False
        
        for back_idx in matched_back_indices:
            back_fruit = back_fruits[back_idx]
            back_x, back_y = back_fruit["center"]
            transformed_point = cv2.perspectiveTransform(
                np.array([[[back_x, back_y]]], dtype=np.float32), H
            )
            tx, ty = transformed_point[0][0]
            
            distance = np.sqrt((front_x - tx)**2 + (front_y - ty)**2)
            if distance < 30:
                matched = True
                break
        
        if not matched:
            color_counts[front_color] += 1
    
    return color_counts

def main():
    # Paths configuration
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    # Count fruits
    counts = count_fruits(front_path, back_path, model_path)
    
    # Display results
    print("\nFinal Fruit Counts:")
    for color, count in counts.items():
        if count > 0:
            print(f"{color.capitalize()}: {count}")

if __name__ == "__main__":
    main()


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Front4.jpg: 480x640 1 2, 1 3, 442.5ms
Speed: 8.0ms preprocess, 442.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\4\Back4.jpg: 480x640 1 3, 512.5ms
Speed: 0.0ms preprocess, 512.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

Final Fruit Counts:
Green: 1


In [33]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_and_get_colors(img_path, model, img):
    """Detect fruits and get their colors in one pass"""
    fruits = []
    for box in model.predict(img_path)[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        
        # Get color
        roi = img[y1:y2, x1:x2]
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        hue = np.mean(hsv[:, :, 0])
        
        # Determine color
        if hue < 15 or hue > 160: color = "red"
        elif 35 <= hue < 85: color = "green"
        elif 85 <= hue < 100: color = "blue"
        elif 100 <= hue < 160: color = "purple"
        else: color = "unknown"
        
        fruits.append({
            "center": ((x1 + x2) // 2, (y1 + y2) // 2),
            "color": color
        })
    return fruits

def count_fruits(front_path, back_path, model_path):
    # Initialize
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    # Get detections with colors
    front_fruits = detect_and_get_colors(front_path, model, front_img)
    back_fruits = detect_and_get_colors(back_path, model, back_img)
    
    # Compute homography
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    matches = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True).match(des1, des2)
    
    good_matches = sorted(matches, key=lambda x: x.distance)[:50]
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    
    # Count fruits
    counts = {"red": 0, "green": 0, "blue": 0, "purple": 0, "unknown": 0}
    matched_back = set()
    
    # Transform all back centers at once
    back_centers = np.float32([[f["center"]] for f in back_fruits])
    if len(back_centers) > 0:
        transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2)
        
        # Match fruits
        for front in front_fruits:
            front_center = np.array(front["center"])
            matched = False
            
            for i, trans_center in enumerate(transformed_centers):
                if i not in matched_back:
                    dist = np.linalg.norm(front_center - trans_center)
                    if dist < 30 and front["color"] == back_fruits[i]["color"]:
                        matched_back.add(i)
                        matched = True
                        break
            
            if not matched:
                counts[front["color"]] += 1
    
    # Add unmatched back fruits
    for i, back in enumerate(back_fruits):
        if i not in matched_back:
            counts[back["color"]] += 1
    
    return counts

if __name__ == "__main__":
    counts = count_fruits(
        r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg',
        r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg',
        r'C:\Users\saart\OneDrive\Desktop\best.pt'
    )
    
    print("\nFinal Fruit Counts:")
    for color, count in counts.items():
        if count > 0:
            print(f"{color.capitalize()}: {count}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg: 480x640 5 2s, 3 3s, 635.1ms
Speed: 0.0ms preprocess, 635.1ms inference, 8.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg: 480x640 8 2s, 3 3s, 560.6ms
Speed: 5.3ms preprocess, 560.6ms inference, 6.9ms postprocess per image at shape (1, 3, 480, 640)

Final Fruit Counts:
Green: 4
Blue: 13
